In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# align subworkflow
import nlppln

wf = nlppln.WorkflowGenerator()
wf.load(steps_dir='../cwl/')
wf.load(step_file='../../edlib-align/align.cwl')
print(wf.list_steps())

gs = wf.add_inputs(gs='File[]')
ocr = wf.add_inputs(ocr='File[]')
align_metadata = wf.add_inputs(align_m='string?')
align_changes = wf.add_inputs(align_c='string?')

changes_files, metadata_files = wf.align(file1=ocr, file2=gs, scatter=['file1', 'file2'], scatter_method='dotproduct')
metadata = wf.merge_json(in_files=metadata_files, name=align_metadata)
changes = wf.merge_json(in_files=changes_files, name=align_changes)

alignment_files = wf.char_align(ocr_text=ocr, gs_text=gs, metadata=metadata_files, 
                                scatter=['ocr_text', 'gs_text', 'metadata'], scatter_method='dotproduct')

wf.add_outputs(metadata=metadata, changes=changes, alignments=alignment_files)
wf.save('../cwl/align-texts-wf.cwl')

In [ ]:
import nlppln

wf = nlppln.WorkflowGenerator()
wf.load(steps_dir='../cwl/')
print(wf.list_steps())

in_dir = wf.add_inputs(in_dir='Directory')
ocr_dir_name = wf.add_inputs(ocr_dir_name='string')
gs_dir_name = wf.add_inputs(gs_dir_name='string')
aligned_dir_name = wf.add_inputs(aligned_dir_name='string')
ocr_char_counts = wf.add_inputs(ocr_n='string?')
gs_char_counts = wf.add_inputs(gs_n='string?')
align_metadata = wf.add_inputs(align_m='string?')
align_changes = wf.add_inputs(align_c='string?')

#dncvu_files = wf.ls(in_dir=in_dir)
vudnc_files = wf.vudnc_select_files(in_dir=in_dir)
gs_with_empty, ocr_with_empty = wf.vudnc2ocr_and_gs(in_file=vudnc_files, scatter='in_file', scatter_method='dotproduct')
gs, ocr = wf.remove_empty_files(gs_files=gs_with_empty, ocr_files=ocr_with_empty)

gs_dir = wf.save_files_to_dir(dir_name=gs_dir_name, in_files=gs)
ocr_dir = wf.save_files_to_dir(dir_name=ocr_dir_name, in_files=ocr)

ocr_cc = wf.count_chars(in_file=ocr, scatter='in_file', scatter_method='dotproduct')
ocr_cc_csv = wf.merge_json(in_files=ocr_cc, name=ocr_char_counts)

gs_cc = wf.count_chars(in_file=gs, scatter='in_file', scatter_method='dotproduct')
gs_cc_csv = wf.merge_json(in_files=gs_cc, name=gs_char_counts)

char_alignment_files, changes, metadata = wf.align_texts_wf(gs=gs, ocr=ocr, align_c=align_changes, align_m=align_metadata)
aligned_dir = wf.save_files_to_dir(dir_name=aligned_dir_name, in_files=char_alignment_files)

wf.add_outputs(gs_dir=gs_dir, ocr_dir=ocr_dir, ocr_char_counts=ocr_cc_csv, gs_char_counts=gs_cc_csv, metadata=metadata, changes=changes)
wf.add_outputs(aligned_dir=aligned_dir)
wf.save('../cwl/vudnc-preprocess.cwl')